In [1]:
import sys
sys.path.append("../src")

In [8]:
import os
import json
from langchain_openai import ChatOpenAI
from langchain.prompts.chat import (
    ChatPromptTemplate,
    SystemMessagePromptTemplate,
    HumanMessagePromptTemplate,
)
from langchain.prompts import PromptTemplate
from dotenv import load_dotenv
from langchain.chains import LLMChain, SequentialChain
from langchain.output_parsers import PydanticOutputParser, OutputFixingParser
from pydantic import BaseModel, Field

from prompts import SYSTEM_MESSAGE, HUMAN_MESSAGE_EXT_DATA, EMAIL_CONTENT_SAMPLE, HUMAN_MESSAGE_GEN_MAIL, all_info_ext_req
from data_class import AllInfo, AGT, Customer

load_dotenv(verbose=True)
dotenv_path = os.path.join('../.env')
load_dotenv(dotenv_path)
OPENAI_API_KEY = os.environ.get("OPENAI_API_KEY")
OPENAI_MODEL = os.environ.get("OPENAI_MODEL")
print("using model:", OPENAI_MODEL)

using model: gpt-3.5-turbo


In [3]:
all_info_parser = PydanticOutputParser(pydantic_object=AllInfo)

In [4]:
system_message_prompt_templ = SystemMessagePromptTemplate.from_template(SYSTEM_MESSAGE)
ext_data_human_message_templ = HumanMessagePromptTemplate(
    prompt=PromptTemplate(
        template=HUMAN_MESSAGE_EXT_DATA,
        input_variables=["email_content", "request"],
        partial_variables={"format_instructions": all_info_parser.get_format_instructions()}
    )
)
ext_data_chat_prompt = ChatPromptTemplate.from_messages([system_message_prompt_templ, ext_data_human_message_templ])
ext_data_chain = LLMChain(llm=ChatOpenAI(model_name=OPENAI_MODEL), prompt=ext_data_chat_prompt)

gen_mail_human_message_templ = HumanMessagePromptTemplate(
    prompt=PromptTemplate(
        template=HUMAN_MESSAGE_GEN_MAIL,
        input_variables=["email_content", "lacked_info"],
    )
)
gen_mail_chat_prompt = ChatPromptTemplate.from_messages([system_message_prompt_templ, gen_mail_human_message_templ])
gen_mail_chain = LLMChain(llm=ChatOpenAI(model_name=OPENAI_MODEL), prompt=gen_mail_chat_prompt)

In [9]:
def list_none_attributes(instance):
    none_attributes = {}
    for field in instance.__fields__:
        value = getattr(instance, field)
        if value is None:
            none_attributes[field] = None
        elif isinstance(value, (AGT, Customer)):
            none_attrs = list_none_attributes(value)
            none_attributes[field] = none_attrs
    return none_attributes

In [5]:
all_ext_data_chain = SequentialChain(chains=[ext_data_chain], input_variables=["email_content", "request"], verbose=True)
ext_data_ans = all_ext_data_chain({'email_content':EMAIL_CONTENT_SAMPLE, 'request':all_info_ext_req})
out = ext_data_ans['text'].replace("None", "null")
fix_parser = OutputFixingParser.from_llm(parser=all_info_parser, llm=ChatOpenAI(model_name=OPENAI_MODEL))
ext_data = fix_parser.parse(out)

/Users/s-shiinoki/Library/Caches/pypoetry/virtualenvs/pj-xps-trvint-app-q11VC4XD-py3.12/lib/python3.12/site-packages/langchain_core/_api/deprecation.py:117: LangChainDeprecationWarning: The function `__call__` was deprecated in LangChain 0.1.0 and will be removed in 0.2.0. Use invoke instead.
  warn_deprecated(




> Entering new SequentialChain chain...

> Finished chain.


In [16]:
none_attributes = list_none_attributes(ext_data)


def create_none_attr_list_str(none_attributes):
    output = ''
    if len(none_attributes['intermediate_company_info']):
        output += 'from intermediate company information:\n'
        for k in none_attributes['intermediate_company_info'].keys():
            output += f'- {k}\n'
    if len(none_attributes['customer_info']):
        output += 'from customer information:\n'
        for k in none_attributes['customer_info'].keys():
            output += f'- {k}\n'
    return output

none_str = create_none_attr_list_str(none_attributes)
print(none_str)

from customer information:
- first_name
- family_name
- nationality
- relationships_between_customers
- budget
- first_time_visitor
- preferences_in_cities
- areas_of_interests
- accommodations
- transportation
- guide



In [18]:
all_gen_mail_chain = SequentialChain(chains=[gen_mail_chain], input_variables=["email_content", "lacked_info"], verbose=True)
gen_mail_ans = all_gen_mail_chain({'email_content':EMAIL_CONTENT_SAMPLE, 'lacked_info': none_str})
generated_mail = gen_mail_ans['text']
print(generated_mail)



> Entering new SequentialChain chain...

> Finished chain.
Subject: Missing Information Required for Travel Inquiry

Dear Estelle Torp,

I hope this email finds you well. Thank you for reaching out regarding the upcoming travel booking for a family trip to Japan from May 20th to June 4th.

In order to provide the best possible travel experience for your clients, I kindly request the following missing information:

1. First Name and Family Name of the travelers
2. Nationality of the travelers
3. Relationships between the customers (e.g., family, friends)
4. Budget for the trip
5. Is this the family's first time visiting Japan?
6. Preferences in cities to visit
7. Areas of interests or specific attractions they would like to explore
8. Preferred accommodations (e.g., hotels, ryokans)
9. Transportation preferences within Japan
10. Need for a guide or any specific language requirements

Having this information will allow us to tailor the travel itinerary to meet the specific needs and pr